# BPSD: Audio Modifications

- Copy audio files from 0_RawData/audio_ripped to 1_Audio
- apply structural modifications if necessary

Johannes Zeitler (johannes.zeitler@audiolabs-erlangen.de), 2024

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from libfmp import b

from scipy.io import wavfile
import librosa

from copy import deepcopy

import pandas as pd

from tqdm.notebook import tqdm

import numbers
import shutil

In [2]:
wav_dir = os.path.join("../", "0_RawData", "audio_ripped")

ann_mod_dir = os.path.join("../", "2_Annotations", "ann_audio_modifications")
wav_out_dir = os.path.join("../", "1_Audio")

logfile = "./mods_list.txt"

In [3]:
annotationsList = [f[:-4] for f in os.listdir(ann_mod_dir) if ".csv" in f]

In [4]:
annotationsList.sort()
for i, l in enumerate(annotationsList):
    print("%i: %s"%(i,l))

0: Beethoven_Op002No1-01_AB96
1: Beethoven_Op002No1-01_FJ62
2: Beethoven_Op002No1-01_JJ90
3: Beethoven_Op002No1-01_VA81
4: Beethoven_Op002No2-01_MB97
5: Beethoven_Op002No2-01_WK64
6: Beethoven_Op002No3-01_FJ62
7: Beethoven_Op002No3-01_WK64
8: Beethoven_Op010No2-01_MB97
9: Beethoven_Op010No2-01_MC22
10: Beethoven_Op010No3-01_WK64
11: Beethoven_Op013-01_WK64
12: Beethoven_Op028-01_AS35
13: Beethoven_Op028-01_AS35_old
14: Beethoven_Op028-01_FJ62
15: Beethoven_Op028-01_WK64
16: Beethoven_Op031No1-01_WK64
17: Beethoven_Op031No2-01_WK64
18: Beethoven_Op031No3-01_WK64
19: Beethoven_Op053-01_WK64
20: Beethoven_Op078-01_FJ62
21: Beethoven_Op078-01_WK64
22: Beethoven_Op079-01_FJ62
23: Beethoven_Op079-01_WK64
24: Beethoven_Op106-01_WK64
25: Beethoven_Op111-01_WK64


In [5]:
def writeLog(s, newline="\n"):
    with open(logfile, "a") as lf:
        lf.write(s+newline)

### get list of all raw audio files

In [8]:
all_wav_dirs = [d for d in os.listdir(wav_dir) if os.path.isdir(os.path.join(wav_dir, d))]
all_wav_dirs.sort()

### copy & modify the audio

In [ ]:
for performer_dir in all_wav_dirs:
    movements = [f for f in os.listdir(os.path.join(wav_dir, performer_dir)) if "-01" in f]
    
    print("Processing %s, found %i movements"%(performer_dir, len(movements)))
    
    for piece in tqdm(movements):
        audioIn, Fs = librosa.load(os.path.join(wav_dir, performer_dir, piece), sr=22050, mono=True)
        audioOut = deepcopy(audioIn)
        
        # modify piece if necessary
        if piece[:-4] in annotationsList:
            print("modifying %s"%(piece[:-4]))
            parts = piece[:-4].split("_")
            sonata = parts[0]+"_"+parts[1]+"_" + parts[2]

            ann_mod = b.b_annotation.read_csv(os.path.join(ann_mod_dir, piece[:-4]+".csv"), header=True)

            mods_list = []

            for _, row in ann_mod.iterrows():
                mod = row.modification

                if mod == "cut":
                    effect_time = row.start        
                    mods_list.append([effect_time, row])    

                elif mod == "copy": 
                    effect_time = row.insert_point        
                    mods_list.append([effect_time, row, audioIn[int(float(row.start)*Fs):int(float(row.end)*Fs)]])

                elif mod == "silence":
                    effect_time = row.insert_point        
                    mods_list.append([effect_time, row])
            


            deltaT = 0
            for mod_row in mods_list:

                mod = mod_row[1].modification

                if mod == "cut":
                    cutStart = mod_row[1].start
                    cutEnd = mod_row[1].end

                    print("cutting [%.2fs - %.2fs]"%(cutStart, cutEnd))
                    writeLog("cutting [%.2fs - %.2fs]"%(cutStart, cutEnd))

                    audioOut = np.concatenate([audioOut[:int( (cutStart+deltaT)*Fs)], audioOut[int( (cutEnd+deltaT)*Fs):]])        
                    deltaT -= (cutEnd - cutStart)

                if mod == "copy":
                    insertPoint = mod_row[0]

                    print("copying [%.2fs - %.2fs] to %.2fs"%(float(mod_row[1].start), float(mod_row[1].end),insertPoint))
                    writeLog("copying [%.2fs - %.2fs] to %.2fs"%(float(mod_row[1].start), float(mod_row[1].end),insertPoint))


                    audioOut = np.concatenate([audioOut[:int( (insertPoint + deltaT)*Fs)], mod_row[2], audioOut[int( (insertPoint+deltaT)*Fs):]])
                    deltaT += len(mod_row[2])/Fs

                if mod == "silence":
                    insertPoint = mod_row[0]
                    silence_duration = float(mod_row[1].silence_duration)

                    print("inserting %.2fs silence at %.2fs"%(silence_duration, insertPoint))
                    writeLog("inserting %.2fs silence at %.2fs"%(silence_duration, insertPoint))


                    audioOut = np.concatenate([audioOut[:int( (insertPoint + deltaT)*Fs)], np.zeros(int(silence_duration*Fs)), audioOut[int( (insertPoint+deltaT)*Fs):]])

                    deltaT += silence_duration
          
            writeLog("")
        ##############################################
        audioOut /= np.max(np.abs(audioOut))

        audioOut *= np.iinfo(np.int16).max


        wavfile.write(filename=os.path.join(wav_out_dir, piece[:-4]+".wav"), rate=22050, data=audioOut.astype(np.int16)) 